In [29]:
import pandas as pd
import re

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [30]:
categoria = 'trucha'#'mezcla' #'cheesecake' #'mezcla pastel' #'mezcla torta' #'postres'
porcentaje_raro = 0.05
stopitem = [] #['molde', 'tenedor', 'modelo', 'plato', 'cuchara', 'bowl', 'set', 'estuche', 'decorativo']

In [31]:
df = pd.read_csv('scraping_de_exito_de_{}.csv'.format(categoria))
df.dropna(inplace = True)

df['precio'] = df['precio'].str.replace('.', '').str.replace(',', '.')
df['precio'] = df['precio'].apply(lambda x: re.sub('[^0-9]', '', x)).astype(float)

df['unidad'] = df['precio_por_gramo'].apply(lambda x: stemmer.stem(str(x).split(' ')[0].lower()))
unidades_raras = df.unidad.value_counts()[df.unidad.value_counts(normalize = True) < porcentaje_raro].index
df = df[~df.unidad.isin(unidades_raras)].copy()

df['precio_unitario'] = df['precio_por_gramo'].apply(lambda x: str(x).split(' ')[-1])
df['precio_unitario'] = df['precio_unitario'].str.replace('.', '').str.replace(',', '.').astype(float)

df = df.drop(['precio_por_gramo'], axis=1).copy()

df['nombre'] = df['nombre'].str.lower()

Q1 = df['precio'].quantile(0.25)
Q3 = df['precio'].quantile(0.75)
IQR = Q3 - Q1
filter = (df['precio'] >= Q1 - 3 * IQR) & (df['precio'] <= Q3 + 3*IQR)
df = df.loc[filter].copy()

correct_items = df['nombre'].apply(lambda x: len([word for word in str(x).split(' ') 
                                                  if stemmer.stem(word) in stopitem])) == 0


stopitem = [stemmer.stem(item) for item in stopitem]

df = df[correct_items].copy()

df.to_csv('datos_de_exito_de_{}.csv'.format(categoria), index = False)

C:\Users\david\AppData\Local\Temp\ipykernel_15920\4059212505.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['precio'] = df['precio'].str.replace('.', '').str.replace(',', '.')
C:\Users\david\AppData\Local\Temp\ipykernel_15920\4059212505.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['precio_unitario'] = df['precio_unitario'].str.replace('.', '').str.replace(',', '.').astype(float)


AttributeError: Can only use .str accessor with string values!

In [24]:
df.head()

,nombre,precio,fecha,lugar,unidad,precio_unitario
0,NaN,32320.0,2022-07-25,Bogotá,gram,32.32
1,NaN,240768.0,2022-07-25,Bogotá,kilogram,40.13
2,NaN,31880.0,2022-07-25,Bogotá,gram,31.88
3,NaN,22800.0,2022-07-25,Bogotá,kilogram,22.80
4,NaN,177525.0,2022-07-25,Bogotá,kilogram,35.51


In [25]:
df.tail()

,nombre,precio,fecha,lugar,unidad,precio_unitario
172,NaN,177525.0,2022-07-25,Villavicencio,kilogram,35.51
173,NaN,23000.0,2022-07-25,Villavicencio,gram,32.86
174,NaN,101349.0,2022-07-25,Villavicencio,kilogram,22.52
175,NaN,38320.0,2022-07-25,Villavicencio,kilogram,38.32
176,NaN,232800.0,2022-07-25,Zipaquira,nan,NaN
